# Inferencia

En este notebook se realiza la inferencia de un modelo previamente entrenado en la clasificación de neumonía y no neumonía.

In [10]:
import numpy as np
import pandas as pd
import pydicom
%matplotlib inline
import matplotlib.pyplot as plt
import os

import keras 
from keras.models import model_from_json


In [11]:
# Again we will see the data
print(os.listdir('./'))

['test5.dcm', 'my_model.json', 'FDA_Submission_Template.md', 'Build and train model.ipynb', 'xray_class_my_model.best.hdf5', '.ipynb_checkpoints', 'test4.dcm', 'sample_labels.csv', 'Inference.ipynb', 'test6.dcm', 'test2.dcm', 'test3.dcm', 'EDA.ipynb', 'test1.dcm']


In [12]:
# This function reads in a .dcm file, checks the important fields for our device, and returns a numpy array
# of just the imaging data
def check_dicom(filename): 
    # todo
    
    print('Load file {} ...'.format(filename))
    ds = pydicom.dcmread(filename)  
    
    if (ds.BodyPartExamined !='CHEST') or (ds.Modality !='DX') or (ds.PatientPosition not in ['AP','PA']):
        img = None
    else:
        print('Load file {} ...'.format(filename))
        img = ds.pixel_array
        
    return img
    
    
# This function takes the numpy array output by check_dicom and 
# runs the appropriate pre-processing needed for our model input
def preprocess_image(img,img_mean,img_std,img_size): 
    # todo
    #print(img.shape)
    
    img = (1/img_mean) * img
    proc_img = np.zeros(img_size)

    proc_img = np.resize(proc_img,img_size)
    return proc_img

# This function loads in our trained model w/ weights and compiles it 
def load_model(model_path, weight_path):
    # load json and create model
    json_file = open(model_path, 'r')
    loaded_model_json = json_file.read()
    json_file.close()
    loaded_model = model_from_json(loaded_model_json)
    
    # load weights into new model
    loaded_model.load_weights(weight_path)
    print("Loaded model from disk")

    return loaded_model

# This function uses our device's threshold parameters to predict whether or not
# the image shows the presence of pneumonia using our trained model
def predict_image(model, img, thresh): 
    # make a prediction
    prediction = model.predict(img)    
       
    if(prediction[0] > thresh):
        prediction = "Pneumonia" 
    else:
        prediction = "No Pneumonia" 
    
    return prediction 

In [13]:
test_dicoms = ['test1.dcm','test2.dcm','test3.dcm','test4.dcm','test5.dcm','test6.dcm']

model_path = 'my_model.json'
weight_path = 'xray_class_my_model.best.hdf5'

IMG_SIZE=(1,224,224,3) # This might be different if you did not use vgg16
img_mean = 0.5
img_std = 255

my_model = load_model(model_path, weight_path)
thresh = 0.005

# use the .dcm files to test your prediction
for i in test_dicoms:
    
    img = np.array([])
    img = check_dicom(i)
    
    if img is None:
        continue
     
    #print(img.shape())
    img_proc = preprocess_image(img,img_mean,img_std,IMG_SIZE)
    pred = predict_image(my_model,img_proc,thresh)
    print(pred)

Loaded model from disk
Load file test1.dcm ...
Load file test1.dcm ...
Pneumonia
Load file test2.dcm ...
Load file test2.dcm ...
Pneumonia
Load file test3.dcm ...
Load file test3.dcm ...
Pneumonia
Load file test4.dcm ...
Load file test5.dcm ...
Load file test6.dcm ...
